<div style="background-color: #4b3832; padding: 20px; border-radius: 10px; box-shadow: 5px 5px 10px #888888;">
  <h1 style="color: white; font-size: 30px; font-weight: bold; text-align: center; text-shadow: 2px 2px 4px #000000;">Problem Statement-3: Data Ingestion
</h1>
</div>

# Importing Libraries

In [1]:
import mysql.connector
from mysql.connector import Error
import pandas as pd
import logging

# Connecting to MySQL Server

In [2]:
def create_server_connection(host_name, user_name, user_password,port_no):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            password=user_password,
            port=port_no
        )
        print("MySQL server connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

connection=create_server_connection("localhost","root","Mysqlroot1@3$",3307)

MySQL server connection successful


# Creating a Database

In [3]:
def create_database(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Database created successfully")
    except Error as err:
        print(f"Error: '{err}'")

query= "CREATE DATABASE sports_database"
create_database(connection,query)

Database created successfully


# Connecting to MySQL Database

In [4]:
def create_db_connection(host_name, user_name, user_password, db_name, port_no):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name,
            port=port_no
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

connection=create_db_connection("localhost","root","Mysqlroot1@3$","sports_database",3307)

MySQL Database connection successful


# Creating Query Execution Function

In [5]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")

# Creating Tables

In [6]:
def create_tables(connection):
    tables = {
        "Player-info": "CREATE TABLE Playerinfo (SerialID INT PRIMARY KEY, Player VARCHAR(60), Team VARCHAR(60), Position VARCHAR(60), Season INT)",
        "Player-demographics": "CREATE TABLE Playerdemographics (SerialID INT, Age INT NOT NULL, Height FLOAT, Weight FLOAT, FOREIGN KEY(SerialID) REFERENCES Playerinfo(SerialID))",
        "Player-stats": "CREATE TABLE Playerstats (SerialID INT, Goals INT, Assists INT, YellowCards INT, RedCards INT, PassCompletionRate FLOAT, DistanceCovered FLOAT, Sprints INT, ShotsOnTarget INT, TacklesWon INT, CleanSheets INT, FOREIGN KEY(SerialID) REFERENCES Playerinfo(SerialID))",
        "Player-training_info": "CREATE TABLE Playertraininginfo (SerialID INT, TrainingHours FLOAT, EffectiveTraining FLOAT, FOREIGN KEY(SerialID) REFERENCES Playerinfo(SerialID))",
        "Player-performance": "CREATE TABLE Playerperformance (SerialID INT, PressurePerformanceImpact FLOAT, MatchPressure INT, FOREIGN KEY(SerialID) REFERENCES Playerinfo(SerialID))",
        "Player-health": "CREATE TABLE Playerhealth (SerialID INT, InjuryHistory INT, PlayerFatigue FLOAT, FatigueInjuryCorrelation FLOAT, FOREIGN KEY(SerialID) REFERENCES Playerinfo(SerialID))"
    }
    for table_name, table_query in tables.items():
        execute_query(connection, table_query)
create_tables(connection)

Query successful
Query successful
Query successful
Query successful
Query successful
Query successful


# Ingestion of Data

In [7]:
def insert_values(connection, table_name, data):
    cursor = connection.cursor()
    for i, row in data.iterrows():
        columns = ', '.join(row.index)
        values_template = ', '.join(['%s'] * len(row))
        query = f"INSERT INTO {table_name} ({columns}) VALUES ({values_template})"
        cursor.execute(query, tuple(row))
    connection.commit()

In [8]:
data = pd.read_csv("new_sports.csv")

## Creation of log file

In [9]:
logging.basicConfig(filename='ingestion_info.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
def perform_ingestion():
    try:
        # Inserting values into the database
        logging.info("Ingestion process started")
        insert_values(connection, "Playerinfo", data[['SerialID', 'Player', 'Team', 'Position', 'Season']])
        insert_values(connection, "Playerdemographics", data[['SerialID', 'Age', 'Height', 'Weight']])
        insert_values(connection, "Playerstats", data[['SerialID', 'Goals', 'Assists', 'YellowCards', 'RedCards', 'PassCompletionRate', 'DistanceCovered', 'Sprints', 'ShotsOnTarget', 'TacklesWon', 'CleanSheets']])
        insert_values(connection, "Playertraininginfo", data[['SerialID', 'TrainingHours', 'EffectiveTraining']])
        insert_values(connection, "Playerperformance", data[['SerialID', 'PressurePerformanceImpact', 'MatchPressure']])
        insert_values(connection, "Playerhealth", data[['SerialID', 'InjuryHistory', 'PlayerFatigue', 'FatigueInjuryCorrelation']])
        logging.info("Ingestion process completed successfully")
    except Exception as e:
        logging.error(f"Error during ingestion process: {str(e)}")

perform_ingestion()